<a href="https://colab.research.google.com/github/SSANTHOSHXIB/Sample_Chatbot/blob/main/samplechatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [171]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [172]:
with open('/content/intents.json') as file:
    data = json.load(file)

In [173]:
training_sentences = []
training_labels = []
labels = []
responses = []

In [174]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [175]:
num_classes = len(labels)

In [176]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [177]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

In [178]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [179]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [180]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [181]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_1  (None, 16)                0         
 1 (GlobalAveragePooling1D)                                      
                                                                 
 dense_29 (Dense)            (None, 16)                272       
                                                                 
 dense_30 (Dense)            (None, 16)                272       
                                                                 
 dense_31 (Dense)            (None, 31)                527       
                                                                 
Total params: 17071 (66.68 KB)
Trainable params: 17071 (66.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________

In [160]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
4/4 [==============================] - 1s 6ms/step - loss: 3.4344 - accuracy: 0.0531
Epoch 2/500
4/4 [==============================] - 0s 7ms/step - loss: 3.4317 - accuracy: 0.0531
Epoch 3/500
4/4 [==============================] - 0s 10ms/step - loss: 3.4294 - accuracy: 0.0531
Epoch 4/500
4/4 [==============================] - 0s 8ms/step - loss: 3.4273 - accuracy: 0.0531
Epoch 5/500
4/4 [==============================] - 0s 7ms/step - loss: 3.4254 - accuracy: 0.0531
Epoch 6/500
4/4 [==============================] - 0s 6ms/step - loss: 3.4235 - accuracy: 0.0531
Epoch 7/500
4/4 [==============================] - 0s 6ms/step - loss: 3.4211 - accuracy: 0.0531
Epoch 8/500
4/4 [==============================] - 0s 7ms/step - loss: 3.4187 - accuracy: 0.0531
Epoch 9/500
4/4 [==============================] - 0s 7ms/step - loss: 3.4160 - accuracy: 0.0531
Epoch 10/500
4/4 [==============================] - 0s 6ms/step - loss: 3.4141 - accuracy: 0.0531
Epoch 11/500
4/4 [==========

In [161]:
model.save("chat_model")

In [162]:
import pickle

to save the fitted tokenizer

In [163]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

to save the fitted label encoder

In [164]:
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [165]:
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

In [166]:
!pip install colorama

In [167]:
import colorama
colorama.init()
from colorama import Fore, Style, Back

In [168]:
import random
import pickle

In [169]:
with open("intents.json") as file:
    data = json.load(file)

In [170]:
def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break
        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])
        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

In [ ]:
print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: useless


1/1 [==============================] - 0s 91ms/step
ChatBot: Please mail your suggestions to ssan04391@gmail.com. Thank you for helping me improve!
User: 